## 🗓️ `EventPlanningAPI`

### In-memory database architecture

```
    key: {
        field: value,
        ...,
        field: value
    }
```

where we define `key` as a date in YYYY-MM-DD format, `field` as an event hash id, and `value` as an `Event` instance and time-to-live.

In [750]:
import datetime
import random
from typing import Dict, List, Union, Optional

WEEKDAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

DEFAULT_STATE = {
    "_records": {
        datetime.datetime.now().date(): [
            -1322510522584581004, -2456134008116096979
        ]
    },
    "_events": {
        -1322510522584581004: Event(
                'Coffee daily doze ☕️',
                datetime.datetime.now(),
                datetime.timedelta(seconds=1)
            ),
        -2456134008116096979: Event(
                'Study sesh 📚',
                datetime.datetime.now(),
                datetime.timedelta(hours=2)
            )
    },
    "event_counter": 2,
    "name": None
}


class Event:
    """
    A class representing an event in the Calendar API.
    
    This class provides methods for event modification, including notifying guests and updating status of the event.
    
    Attributes: 
        title (str): The name or title of the event.
        start (datetime.datetime): The starting date and time of the event.
        ttl (datetime.timedelta): The time duration for which the event will last.
        end (datetime.datetime): The calculated end date and time of the event.
        eid (int): A unique identifier of the event, assigned only after adding this event to the Calendar instance.
        _status (str): The current status of the event ('Upcoming', 'Current', or 'Completed').
        _guests (List[str]): A list of guest emails associated with the event.
    
    Methods:
        get_guests(): Returns the guest list for the event.
        get_status(): Returns the current status of the event.
        set_event_id(eid: int): Assigns a unique event identifier after adding the event to the calendar.
        update_status(): Updates the event's status based on the current date and time.
        add_guest(guest_contact: str): Adds a guest's email to the event's guest list.
        remove_guest(guest_contact: str): Removes a guest's email from the guest list if present.
        notify_guests(): Sends notification messages to all guests in the event's guest list.

    """
    def __init__(self, title, start, ttl):
        """
        Initialize an Event instance.
        """
        self.title: str = title
        self.start: datetime.datetime = start
        self.ttl: datetime.timedelta = ttl
        self.end: datetime.datetime = start + ttl
        self.eid: int
        self._status: str = None
        self._guests: List[str] = []
    
    def __str__(self):
        """
        An event description.
        
        Returns:
            title (a string): A title of the events.
        """
        print(f'The event "{self.title}" is starting on {self.start}, and ending on {self.end}.')
        print(f'Note that the event happens on {WEEKDAYS[self.start.weekday()]}.')
        if self.get_guests():
            print(f'Guest list: {self.get_guests()}')
        return f'Status: {self._status}.'
    
    def get_guests(self) -> List[str]:
        return self._guests
    
    def get_status(self) -> str:
        return self._status
    
    def set_event_id(self, eid: int) -> None:
        "Set a unique event identifier after adding it to the Calendar instance."
        self.eid = eid
    
    def update_status(self) -> None:
        "Update and assign the event status."
        updated_status = 'Current'
        if datetime.datetime.now() < self.start:
            updated_status = 'Upcoming'
        elif datetime.datetime.now() > self.end:
            updated_status = 'Completed'
        self._status = updated_status
    
    def add_guest(self, guest_contact) -> None:
        "Add guest email to the guests list."
        assert '@' in guest_contact
        self._guests.append(guest_contact)
    
    def remove_guest(self, guest_contact) -> None:
        assert guest_contact in self._guests
        self._guests.remove(guest_contact)
    
    def notify_guests(self) -> None:
        "Notify guests about the upcoming event or any event changes."
        for guest_contact in self.get_guests():
            print(f'The invite was succesfully sent to {guest_contact}!')


class Calendar:
    """
    A calendar system that manages the scheduling and tracking of events, allowing the user to add, cancel, and update events.

    Attributes:
        name (Optional[str]): The owner's name or identifier for the calendar.
        _records (Dict): A dictionary storing event records by date, mapping to lists of event IDs.
        _events (Dict): A dictionary of scheduled events, stored by their unique IDs.
        event_counter (int): The total count of events scheduled in the calendar.

    Methods:
        schedule_event(event: Event, eid: Optional[int] = None): Add an Event instance into the calendar.
        cancel_event(eid: int): Remove a specified event from the calendar.
        update(): Update status of all events with respect to the current time.
        event_history(): Return how many events have been scheduled so far.
        get_upcoming_events(): Return all upcoming events.
        get_current_events(): Return event(s) that is happening right now.
    """
    def __init__(self, name: Optional[str] = None):
        self.name: Optional[str] = name
        self._records: Dict = {}
        self._events: Dict = {}
        self.event_counter: int = 0
        
    def __str__(self) -> str:
        """
        Print the calendar owner's name and all the scheduled events by increasing event date.
        
        Returns:
            current date (a string): Current date in '<month> <day>, <year>' format
        """
        print(f'This is a personal calendar of {self.name}.\n')
        print(f'The current list of events:\n')
        if not self._events:
            print('(empty)')
        else:
            for date in sorted(self._records):
                if not self._records[date]:
                    continue
                print(date.strftime("%B %d, %Y"))
                print('###################')
                for eid in self._records[date]:
                    print(self._events[eid])
                    print('###################')
                print('\n')
        return datetime.datetime.now().strftime("%B %d, %Y")
        
    def schedule_event(self, event: Event, eid: Optional[int] = None) -> None:
        """
        Add an Event instance into the calendar.
        
        The structure of the calendar's:
            records:
                date1: {
                    event1_id, ..., eventN_id
                },
                ...
            events: {
                event1_id: <associated Event instance>,
                ...
            }
        """
        if not eid:
            eid = hash(event.title + str(random.random()))
        event.set_event_id(eid)
        self._events[eid] = event
        if event.start.date() in self._records:
            self._records[event.start.date()].append(eid)
        else:
            self._records[event.start.date()] = [eid]
        self.event_counter += 1    
        
    def cancel_event(self, eid: int) -> bool:
        """
        Remove a specified event from the calendar.
        
        Returns:
            canceled_event (bool): return True if successfully canceled the event; False otherwise.
        """
        if eid not in self._events:
            return False
        event_date = self._events[eid].start.date()
        self._records[event_date].remove(eid)
        del self._events[eid]
        return True
        
    def update(self) -> None:
        """
        Update status of all events with respect to the current time.
        """
        for eid in self._events:
            self._events[eid].update_status()
    
    def event_history(self) -> int:
        """
        Return how many events have been scheduled so far.
        
        Returns:
            event_counter (int): number of events that have ever been added.
        """
        print(f'You had {self.event_counter} events scheduled so far.')
        return self.event_counter
        
    def get_upcoming_events(self) -> Union[Event, List[str]]:
        """
        Return all upcoming events.
        
        Returns:
            upcoming_eids (Union[Event, List[str]]): an event or a list of events' IDs in the calendar that has not started.
        """
        upcoming_eids = []
        for eid in self._events:
            if self._events[eid].get_status() == 'Upcoming':
                upcoming_eids.append(eid)
        if len(upcoming_eids) == 1:
            print(self._events[upcoming_eids[0]])
            return upcoming_eids[0]
        return upcoming_eids
    
    def get_current_events(self) -> Union[Event, List[Event]]:
        """
        Return event(s) that is happening right now.
        
        Returns:
            current_eids (Union[Event, List[str]]): an event or a list of events' IDs in the calendar that are happening right now.
        """
        current_eids = []
        for eid in self._events:
            if self._events[eid].get_status() == 'Current':
                current_eids.append(eid)
        if len(current_eids) == 1:
            print(self._events[current_eids[0]])
            return current_eids[0]
        return current_eids


**Other API notes**
- how to query events while removing them? use hashing
    - what if we want to add events with the same name? add random variable into hashing for unique identifiers

- do we want to add backup and restore methods? when ttl needs to be updated

## Potential MT prompts

Given `DEFAULT_STATE`, ask these in this order:
- Add a 'CURRENTLY HAPPENING' prefix to all currently happening events
- Add halloween into my calendar and show my schedule with updated event status
- Remove halloween from my schedule
- Add halloween back. Also, add thanksgiiving and my upcoming flight on thanksgiving from 8:15 pm to 9:45 pm
- My flight on thanksgiving has been delayed! It now starts an hour later
- My friends are taking the same flight! Add them as guests to my flight. Their emails are gracefulstrawberry and berrycandidwatermelon at berkeley dot edu

Ask model to create/override methods:

- Create a `switch_time_zone` method
- Add a variable for the `Event` class so I could attach a Zoom link to the event

More potential cases:
- Ask model to remove general event type (check if model comes up with a follow-up question specifying which event of those suitable should be removed) 

Note that we must update calendar (i.e. each event status) every time we modify it. For example, if we want to add a new event into the schedule, the correct code must update the calendar first.

## Example

In [775]:
my_calendar = Calendar('sadbanana')

In [776]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

(empty)
November 04, 2024


In [777]:
my_calendar.schedule_event(
    Event('Coffee daily doze ☕️', datetime.datetime.now(), datetime.timedelta(seconds=1)),
    hash('Coffee daily doze ☕️')
)

In [778]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:16:41.729176, and ending on 2024-11-04 00:16:42.729176.
Note that the event happens on Monday.
Status: Current.
###################


November 04, 2024


In [779]:
my_calendar.get_current_events()

The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:16:41.729176, and ending on 2024-11-04 00:16:42.729176.
Note that the event happens on Monday.
Status: Current.


-1322510522584581004

In [780]:
my_calendar.schedule_event(
    Event('Study sesh 📚', datetime.datetime.now(), datetime.timedelta(hours=2)),
    hash('Study sesh 📚')
)

In [781]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:16:41.729176, and ending on 2024-11-04 00:16:42.729176.
Note that the event happens on Monday.
Status: Completed.
###################
The event "Study sesh 📚" is starting on 2024-11-04 00:16:42.043401, and ending on 2024-11-04 02:16:42.043401.
Note that the event happens on Monday.
Status: Current.
###################


November 04, 2024


In [782]:
# check events' id
for eid in list(my_calendar._events.keys()):
    print(eid, my_calendar._events[eid].title)

-1322510522584581004 Coffee daily doze ☕️
-2456134008116096979 Study sesh 📚


After the status of the event changed, modify it in-place.

#### Prompt: "Add a 'CURRENTLY HAPPENING' prefix to all currently happening events"

In [711]:
eids = my_calendar.get_current_events()

The event "Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.


In [712]:
if type(eids) == int:
    my_calendar._events[eids].title = 'CURRENTLY HAPPENING ' + my_calendar._events[eids].title
else:
    for eid in eids:
        my_calendar._events[eid].title = 'CURRENTLY HAPPENING ' + my_calendar._events[eid].title

In [713]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:09:33.089435, and ending on 2024-11-04 00:09:34.089435.
Note that the event happens on Monday.
Status: Completed.
###################
The event "CURRENTLY HAPPENING Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.
###################


November 04, 2024


#### Prompt: "Add halloween into my calendar and show my schedule with updated event status"

In [714]:
# initialize event
halloween = Event('Halloween 🎃', datetime.datetime(2025, 10, 31), datetime.timedelta(1))
halloween.update_status()
print(halloween)

The event "Halloween 🎃" is starting on 2025-10-31 00:00:00, and ending on 2025-11-01 00:00:00.
Note that the event happens on Friday.
Status: Upcoming.


In [715]:
# add event
my_calendar.schedule_event(halloween)

In [716]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:09:33.089435, and ending on 2024-11-04 00:09:34.089435.
Note that the event happens on Monday.
Status: Completed.
###################
The event "CURRENTLY HAPPENING Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.
###################


October 31, 2025
###################
The event "Halloween 🎃" is starting on 2025-10-31 00:00:00, and ending on 2025-11-01 00:00:00.
Note that the event happens on Friday.
Status: Upcoming.
###################


November 04, 2024


In [717]:
# check events' id
for eid in list(my_calendar._events.keys()):
    print(eid, my_calendar._events[eid].title)

1792099008653040614 Coffee daily doze ☕️
-6677544021441104700 CURRENTLY HAPPENING Study sesh 📚
-7433034805700927944 Halloween 🎃


In [718]:
# remove event
my_calendar.cancel_event(list(my_calendar._events.keys())[-1])

True

In [719]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:09:33.089435, and ending on 2024-11-04 00:09:34.089435.
Note that the event happens on Monday.
Status: Completed.
###################
The event "CURRENTLY HAPPENING Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.
###################


November 04, 2024


#### Prompt: "Remove halloween from my schedule"

In [720]:
# add back
my_calendar.schedule_event(halloween)

#### Prompt: "Add halloween back. Also, add thanksgiving and my upcoming flight on thanksgiving from 8:15 pm to 9:45 pm"

In [721]:
# add more events
thanksgiving = Event('Thanksgiving 🦃', datetime.datetime(2024, 11, 28), datetime.timedelta(days=1))
thanksgiving.update_status()
my_calendar.schedule_event(thanksgiving)

flight = Event('Flight to San Diego 🛫', datetime.datetime(2024, 11, 28, 20, 15, 0), datetime.timedelta(hours=1, minutes=30))
flight.update_status()
my_calendar.schedule_event(flight)

In [722]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:09:33.089435, and ending on 2024-11-04 00:09:34.089435.
Note that the event happens on Monday.
Status: Completed.
###################
The event "CURRENTLY HAPPENING Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.
###################


November 28, 2024
###################
The event "Thanksgiving 🦃" is starting on 2024-11-28 00:00:00, and ending on 2024-11-29 00:00:00.
Note that the event happens on Thursday.
Status: Upcoming.
###################
The event "Flight to San Diego 🛫" is starting on 2024-11-28 20:15:00, and ending on 2024-11-28 21:45:00.
Note that the event happens on Thursday.
Status: Upcoming.
###################


October 31, 2025
###################
The event "Halloween 🎃" is starting on 2025-10-3

#### Prompt: "My flight on thanksgiving has been delayed! It now starts an hour later"

In [723]:
# check events' id
for eid in list(my_calendar._events.keys()):
    print(eid, my_calendar._events[eid].title)

1792099008653040614 Coffee daily doze ☕️
-6677544021441104700 CURRENTLY HAPPENING Study sesh 📚
5055264609276513585 Halloween 🎃
1266889921927200035 Thanksgiving 🦃
-6695472132160786868 Flight to San Diego 🛫


In [724]:
# modify event: flight has been rescheduled to an hour later
my_calendar._events[list(my_calendar._events.keys())[-1]].start += datetime.timedelta(hours=1)
my_calendar._events[list(my_calendar._events.keys())[-1]].end += datetime.timedelta(hours=1)

In [725]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:09:33.089435, and ending on 2024-11-04 00:09:34.089435.
Note that the event happens on Monday.
Status: Completed.
###################
The event "CURRENTLY HAPPENING Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.
###################


November 28, 2024
###################
The event "Thanksgiving 🦃" is starting on 2024-11-28 00:00:00, and ending on 2024-11-29 00:00:00.
Note that the event happens on Thursday.
Status: Upcoming.
###################
The event "Flight to San Diego 🛫" is starting on 2024-11-28 21:15:00, and ending on 2024-11-28 22:45:00.
Note that the event happens on Thursday.
Status: Upcoming.
###################


October 31, 2025
###################
The event "Halloween 🎃" is starting on 2025-10-3

#### Prompt: "My friends are taking the same flight! Add them as guests to my flight. Their emails are gracefulstrawberry and berrycandidwatermelon at berkeley dot edu"

In [726]:
# modify event: my friends are now flying with me!
# their emails: 'gracefulstrawberry@berkeley.edu', 'berrycandidwatermelon@berkeley.edu'
my_calendar._events[list(my_calendar._events.keys())[-1]].add_guest('gracefulstrawberry@berkeley.edu')
my_calendar._events[list(my_calendar._events.keys())[-1]].add_guest('berrycandidwatermelon@berkeley.edu')

In [727]:
# notify guests about the event
my_calendar._events[list(my_calendar._events.keys())[-1]].notify_guests()

The invite was succesfully sent to gracefulstrawberry@berkeley.edu!
The invite was succesfully sent to berrycandidwatermelon@berkeley.edu!


In [728]:
# print calendar
my_calendar.update()
print(my_calendar)

This is a personal calendar of sadbanana.

The current list of events:

November 04, 2024
###################
The event "Coffee daily doze ☕️" is starting on 2024-11-04 00:09:33.089435, and ending on 2024-11-04 00:09:34.089435.
Note that the event happens on Monday.
Status: Completed.
###################
The event "CURRENTLY HAPPENING Study sesh 📚" is starting on 2024-11-04 00:09:34.039320, and ending on 2024-11-04 02:09:34.039320.
Note that the event happens on Monday.
Status: Current.
###################


November 28, 2024
###################
The event "Thanksgiving 🦃" is starting on 2024-11-28 00:00:00, and ending on 2024-11-29 00:00:00.
Note that the event happens on Thursday.
Status: Upcoming.
###################
The event "Flight to San Diego 🛫" is starting on 2024-11-28 21:15:00, and ending on 2024-11-28 22:45:00.
Note that the event happens on Thursday.
Guest list: ['gracefulstrawberry@berkeley.edu', 'berrycandidwatermelon@berkeley.edu']
Status: Upcoming.
###################



Guests should also be notified about any changes in the event.

## Minor evaluation runs

#### Gorilla OpenFunctions-v2

```
my_calendar.add_event(Event(title='Halloween', start=datetime.datetime(2023, 10, 31, 19, 00), duration=datetime.timedelta(hours=1))), my_calendar.__str__()
```

<font color="red">
    Gorilla assumed it was 2023, although it is currently 2024. Also, it forgot to update status of the event after adding that event and while outputting calendar.
</font>

In [279]:
my_calendar.add_event(Event(title='Halloween', start=datetime.datetime(2023, 10, 31, 19, 0), duration=datetime.timedelta(hours=3)))

TypeError: Event.__init__() got an unexpected keyword argument 'duration'

In [332]:
my_calendar.__str__()

This is a personal calendar of sadbanana.
The current list of events:
###################
The event "Halloween" is starting on 2023-10-31 19:00:00, and ending on 2023-10-31 22:00:00.
Note that the event happens on Monday.
Status: None.
###################


'November 01, 2024'

#### ChatGPT 4o

```
    import datetime

    # Initialize or retrieve the calendar
    my_calendar = Calendar("sadbanana")

    # Define the Halloween event
    halloween_event = Event(
        title="Halloween Party",
        start=datetime.datetime(datetime.datetime.now().year, 10, 31, 18, 0),  # Halloween on October 31 at 6 PM
        duration=datetime.timedelta(hours=5)  # Let's assume a 5-hour party
    )

    # Add the Halloween event to the calendar
    my_calendar.add_event(halloween_event)

    # Update all event statuses
    my_calendar.update()

    # Display the updated calendar
    print(my_calendar)
```

<font color='green'>
    Correctly updated status of all events before outputting.
</font>

In [348]:
import datetime

# Initialize or retrieve the calendar
my_calendar = Calendar("sadbanana")

# Define the Halloween event
halloween_event = Event(
    title="Halloween Party",
    start=datetime.datetime(datetime.datetime.now().year, 10, 31, 18, 0),  # Halloween on October 31 at 6 PM
    duration=datetime.timedelta(hours=5)  # Let's assume a 5-hour party
)

# Add the Halloween event to the calendar
my_calendar.add_event(halloween_event)

# Update all event statuses
my_calendar.update()

# Display the updated calendar
print(my_calendar)

This is a personal calendar of sadbanana.
The current list of events:
###################
The event "Halloween Party" is starting on 2024-10-31 18:00:00, and ending on 2024-10-31 23:00:00.
Note that the event happens on Wednesday.
Status: Completed.
###################
November 01, 2024


### Follow-up prompt: "Now, add thanksgiving into my schedule"

#### ChatGPT 4o

In [349]:
# Define Thanksgiving event
thanksgiving_start = datetime.datetime(datetime.datetime.now().year, 11, 28, 13, 0)  # Fourth Thursday in November
thanksgiving_duration = datetime.timedelta(hours=6)  # Duration of 6 hours
thanksgiving_event = Event("Thanksgiving", thanksgiving_start, thanksgiving_duration)

# Add the Thanksgiving event to the calendar
my_calendar.add_event(thanksgiving_event)

# Update the status of all events
my_calendar.update()

# Print the updated schedule
print(my_calendar)

This is a personal calendar of sadbanana.
The current list of events:
###################
The event "Halloween Party" is starting on 2024-10-31 18:00:00, and ending on 2024-10-31 23:00:00.
Note that the event happens on Wednesday.
Status: Completed.
###################
###################
The event "Thanksgiving" is starting on 2024-11-28 13:00:00, and ending on 2024-11-28 19:00:00.
Note that the event happens on Wednesday.
Status: Upcoming.
###################
November 01, 2024


### Follow-up prompt: "Remove halloween from my schedule"

#### ChatGPT 4o

<font color='orange'>
    ChatGPT 4o made a good try to define the id of the event by parsing through all the events, but the title of the event was not exactly "Halloween", so it could not remove it.
</font>

In [350]:
# Find and remove Halloween event by title
for eid, event in list(my_calendar._events.items()):
    if event.title == "Halloween":
        my_calendar.remove_event(eid)
        print("Halloween event removed.")

# Update the status of all events
my_calendar.update()

# Print the updated schedule
print(my_calendar)

This is a personal calendar of sadbanana.
The current list of events:
###################
The event "Halloween Party" is starting on 2024-10-31 18:00:00, and ending on 2024-10-31 23:00:00.
Note that the event happens on Wednesday.
Status: Completed.
###################
###################
The event "Thanksgiving" is starting on 2024-11-28 13:00:00, and ending on 2024-11-28 19:00:00.
Note that the event happens on Wednesday.
Status: Upcoming.
###################
November 01, 2024


## Alternative implementations

Using `TTLCache`. See [reference](https://stackoverflow.com/questions/31771286/python-in-memory-cache-with-time-to-live).